#### Steps to setup Openbb SDK in Jupyter notebook

1. Check conda version

    $ conda --version 

2. Update conda recommended

    $ conda update -n base -c conda-forge conda

3. Setup and activate the `meta-labell` environment

    $ conda env create --file=environment.yaml

    $ conda activate meta-labell    

4. Install openbb sdk
    
    $ pip install "openbb" --no-cache-dir              

### Introduction 
This notebook studies fundamentals and price signals of common cryptocurrencies using the OpenBB SDK. We consider the universe of cryptos with at least a $ 100 Million USD market capitalization on Coinmarketcap.  

### Data
We use the OpenBB SDK to download the data from the OpenBB API. The data is stored in a local database. The data is updated daily. 

### Methods 
We begin with a simple forecasting model such as AutoArima and explore intra-day changes in volatility and prices. As part of our token selection process, we also use fundamental knowledge about each cryptocurrency, especially for newer tokens such as the number of developers on each project's Github, and the frequency of commits. 


In [1]:
%load_ext lineapy

In [2]:
import numpy as np
import pandas as pd
from openbb_terminal.sdk import openbb
from statsforecast.models import AutoARIMA
import requests
import re
import altair as alt
from scipy.linalg import svd

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf
%matplotlib inline

# Turn off warnings and messages 
import warnings
warnings.filterwarnings('ignore')

START                                                                           
END                                                                             
START                                                                           
END                                                                             
START                                                                           
END                                                                             


Forecasting Toolkit is disabled. To use the Forecasting features please install the toolkit following the instructions here: https://my.openbb.co/app/sdk/installation

Portfolio Optimization Toolkit is disabled. To use the Optimization features please install the toolkit following the instructions here: https://my.openbb.co/app/sdk/installation

Logging configuration finished                                                  
Logging set to ['file', 'posthog']                                              
Verbosity set to 20                                                             
LOGFORMAT:                                                                      
%(levelname)s-%(appName)s-%(commitHash)s-%(appId)s-%(sessionId)s-%(userId)s-%(as
ctime)s-%(name)s-%(funcName)s-%(lineno)s-%(message)s                            
STARTUP: {"PREFERENCES": {"PLOT_BACKEND": null, "PLOT_DPI": 100, "PLOT_HEIGHT": 
500, "PLOT_WIDTH": 800, "PLOT_HEIGHT_PERCENTAGE": 50.0, "PLOT_WIDTH_PERCENTAGE":
70.0, "PLOT_OPEN_EXPORT": false, "PLOT_ENABLE_PYWRY": true, "PLOT_PYWRY_WIDTH": 
1400, "PLOT_PYWRY_HEIGHT": 762, "FILE_OVERWRITE": false, "SHOW_VERSION": true,  
"RETRY_WITH_LOAD": false, "USE_TABULATE_DF": true, "USE_INTERACTIVE_DF": true,  
"USE_CLEAR_AFTER_CMD": false, "USE_DATETIME": true, "USE_PROMPT_TOOLKIT": true, 
"USE_PLOT_AUTOSCALING": fals

In [ ]:
# Show trending coins
openbb.crypto.disc.trending()

In [ ]:
trades_ethusdt = openbb.crypto.dd.trades(exchange_id = 'coinbaseprime', symbol = 'ETH', to_symbol = 'USDT')

trades_ethusdt.tail(5)

In [ ]:
btc_df = openbb.crypto.load(symbol="btc",to_symbol="usd",start_date="2019-01-01",source="YahooFinance")
btc_df.tail(5)

In [ ]:
# Plot the Adj Close price overtime for BTC
import matplotlib.pyplot as plt

btc_df['Adj Close'].plot(figsize=(10, 7), color='r')
plt.ylabel('BTC Price')
plt.title('BTC Price Over Time')

### References 
Sean Taylor. Gas Price Forecast. https://github.com/seanjtaylor/gas-price-forecast